In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_DIR = "/content/drive/MyDrive/DS340_Final_Project/augmented_spectrograms_dataset"

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, applications, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
IMG_SIZE      = 224
BATCH_SIZE    = 16
NUM_CLASSES   = 8
EPOCHS        = 30


train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.40)

train_gen = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=123)

valtest_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.50)

val_gen = valtest_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    seed=123)

test_gen = valtest_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    seed=123)

Found 2431 images belonging to 8 classes.
Found 2024 images belonging to 8 classes.
Found 2024 images belonging to 8 classes.


In [ ]:
base = applications.ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3))

base.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
inputs  = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation="relu")(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax")(x)

songmodel = tf.keras.Model(inputs, outputs)

In [ ]:
songmodel.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

early_stop = callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True)

In [ ]:
history = songmodel.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stop])


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 1251s 8s/step - accuracy: 0.1774 - loss: 1.9937 - val_accuracy: 0.2070 - val_loss: 1.7627
Epoch 2/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 18s 117ms/step - accuracy: 0.1883 - loss: 1.8279 - val_accuracy: 0.1823 - val_loss: 1.7652
Epoch 3/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 18s 122ms/step - accuracy: 0.2216 - loss: 1.7790 - val_accuracy: 0.2312 - val_loss: 1.7387
Epoch 4/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 18s 120ms/step - accuracy: 0.2174 - loss: 1.7574 - val_accuracy: 0.2851 - val_loss: 1.7211
Epoch 5/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 18s 117ms/step - accuracy: 0.2199 - loss: 1.7728 - val_accuracy: 0.3024 - val_loss: 1.7199
Epoch 6/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 19s 122ms/step - accuracy: 0.2286 - loss: 1.7631 - val_accuracy: 0.2332 - val_loss: 1.7115
Epoch 7/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 18s 120ms/step - accuracy: 0.2218 - loss: 1.7386 - val_accuracy: 0.2475 - val_loss: 1.7087
Epoch 8/30
152/152 ━━━━━━━━━━━━━━━━━━━━ 18s 119ms/step - accuracy: 0.2346 - loss: 1.

In [ ]:
test_loss, test_acc = songmodel.evaluate(test_gen, verbose=0)
print(f"\nTest accuracy (frozen backbone): {test_acc:.4f}")

KeyboardInterrupt: 

In [ ]:
#Unfreeze top layers, compile with tiny learning rate. and train again
base.trainable = True
for layer in base.layers[:-30]:
    layer.trainable = False

songmodel.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"])

history_fine = songmodel.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=val_gen,
    callbacks=[early_stop])

In [ ]:
test_loss, test_acc = songmodel.evaluate(test_gen, verbose=0)
print(f"\nTest accuracy (fine-tuned): {test_acc:.4f}")